<center><h2>Jane Street Market Prediction | EDA with SweetViz | katsu1110 </h2></center><hr>

EDA seems to be vital in this competition, as provided features are anonymous. Given the number of features, let's use [sweetviz](https://pypi.org/project/sweetviz/) to cover all the columns.
This module gives us very nice overview of the provided data.

In [ ]:
!pip install sweetviz

In [ ]:
import janestreet
import numpy as np
import pandas as pd

import os, sys
import gc
import math
import random
import pathlib
from tqdm import tqdm
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn import linear_model
import operator
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm_notebook as tqdm
import sweetviz as sv

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib_venn import venn2
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('fivethirtyeight')
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

### Make prettier the prints ###
from colorama import Fore
c_ = Fore.CYAN
m_ = Fore.MAGENTA
r_ = Fore.RED
b_ = Fore.BLUE
y_ = Fore.YELLOW
g_ = Fore.GREEN

# Config
Some configuration setups.

In [ ]:
SEED = 20201225 # Merry Christmas!
# INPUT_DIR = '../input/jane-street-market-prediction/'
INPUT_DIR = '../input/janestreet-save-as-feather/'
TRADING_THRESHOLD = 0.5 # 0 ~ 1: The smaller, the more aggressive
DATE_BEGIN = 200 # 0 ~ 499: set 0 for model training using the complete data 
USE_SAMPLE_WEIGHT = False

# Load data
I have already saved the training data in the feather-format in [my another notebook](https://www.kaggle.com/code1110/janestreet-save-as-feather?scriptVersionId=47635784). Loading csv takes time but loading feather is really light:)

In [ ]:
%%time

# load data blitz fast!
def load_data(input_dir=INPUT_DIR):
    train = pd.read_feather(pathlib.Path(input_dir + 'train.feather'))
    features = pd.read_feather(pathlib.Path(input_dir + 'features.feather'))
    example_test = pd.read_feather(pathlib.Path(input_dir + 'example_test.feather'))
    ss = pd.read_feather(pathlib.Path(input_dir + 'example_sample_submission.feather'))
    return train, features, example_test, ss

train, features, example_test, ss = load_data(INPUT_DIR)

# Feature 
'feature.csv' is very sparse...I don't know what it means. Let's visualize it for now. We have 29 'tag's (whatever it means) for each 139-feature.

In [ ]:
features = features * 1
print(features.shape)
features.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 12))
sns.heatmap(features[[f for f in features.columns.values.tolist() if 'tag' in f]], ax=ax);
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_yticks(range(features.shape[0]))
ax.set_yticklabels(features['feature'].values)
ax.set_title('features.csv')

It would be great if some of you could decode this data:D

# EDA with SweetViz
The training data is huge, so I sample the data based on 'date'.

In [ ]:
%%time

my_report = sv.analyze(train.groupby('date').apply(lambda x: x.sample(frac=0.1, random_state=SEED)))

In [ ]:
my_report.show_notebook() 

# Takeaways

- 'date' is slightly negatively correlated with some features (such as 'feature_45'), suggesting that some features are decaying overtime.
- 'weight' has median of 1 and has clear positive (e.g. feature_51) and negative (e.g. feature_126) correlations with some features.
- 'resp' has strong positive correlations with other resps (1-4), but 'resp' is not well correlated with features (bad news!).
- 'feature_0' is binary, and highly correlated with some features (e.g. 'feature_40').
- Many 'feature' are highly correlated with one another, suggesting redundancy.
- 'feature_64 ~ 68' are perfectly correlated with another (pearson r = 1)!
- 'ts_id' has negative correlation with some features (same as 'date'), again suggesting that some features decay overtime.

Hope you like this kearnel and learn some!